# Spatial Index Notebook (general) with utilities module

### Goals: (1) Generate and save colonies dataset (2) Create function that creates service index. (3) Apply service index function to all available services (4) Add overall PSI column. (5) Save/ship to colleagues

In this notebook, I will complete the following tasks:
* Load in `colonies_with_neighbors` and process **[DONE]**
    * Merge population data with colonies data
    * Create nbr_dist column
    * Add ndmc_dist column
    * Remove extraneous columns and save as pickle file
* Calculate and add services indices to GeoDataFrame
    * Where needed, convert CSV lat/long to GeoDataFrame with Shapely Points
    * Convert shapefiles to the same coordinate reference system
    * Check that all point coordinates are in Delhi
    * Make sure that no geometry fields are missing
    * Calculate service index and add to GeoDataFrame
* Descriptive statistics for service indices
    * mean, min, max
    * Grouped by settlement type: take average PSI for all colonies with a specific settlement type
    * Grouped by MCD category: take average PSI for all colonies with a specific MCD category
    * Grouped by distance from NDMC? (optional)
* Save service indices and descriptive statistics to .csv file

In [6]:
# Import necessary modules
from itertools import islice
import pickle
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import box, Polygon, Point
from shapely.ops import cascaded_union
from pyproj import CRS
import spatial_index_utils

# Reload spatial_index_utils
reload(spatial_index_utils)

# Constants
# Pseudo Mercator
epsg_code = 3857 

## Process and save colonies shapefile for PSI

In [7]:
# open colonies_with_neighbors file
# This has NDMC+JJC colonies with neighbors of each polygon
# based on other polygons that intersect, cross, touch
# or overlap.

with open('colonies_with_neighbors.data', 'rb') as f:
    colonies = pickle.load(f)

In [8]:
# Reproject to EPSG 3857
colonies = spatial_index_utils.reproject_gdf(colonies, epsg_code)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [9]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,centroid,polygon_neighbors
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...",POINT (1012643.931 1023967.383),"[2183, 1148, 3169, 1644, 1994, 3059]"
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...",POINT (1008357.317 1025258.233),"[1528, 2869, 3134, 1199, 2387]"
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...",POINT (1009663.437 1022804.496),"[2082, 1148, 1152, 2780, 2934]"
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...",POINT (1008902.061 1025094.880),"[1760, 2082, 2869, 3134, 1199, 3025, 2387]"
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...",POINT (1009457.571 1023953.212),"[1276, 1528, 2183, 1148, 3134, 1152, 1199, 302..."


In [10]:
## Calculate distances from polygon centroid to centroids of its neighbors

# Distance should be in meters
# https://en.wikipedia.org/wiki/Easting_and_northing

colonies = spatial_index_utils.calc_nbr_dist(colonies)

In [12]:
# Import 2020 population data
worldpop2020 = pd.read_csv("population_data/pop_colony_wp_2020.csv")

# Restrict dataframe to only two columns:
# layer: population data
# uso_area_u: unique id for colonies
worldpop2020 = worldpop2020[['layer', 'uso_area_u']]

# Merge population data with colonies data
colonies = colonies.merge(worldpop2020, how='inner', 
                          left_on="USO_AREA_U", right_on='uso_area_u')

# Rename 'layer' column as 'population'
colonies = colonies.rename(columns={'layer': 'population'})

# Remove additional column 'uso_area_u'
colonies = colonies.drop(columns=['uso_area_u'])

In [13]:
# Code to generate ndmc_distances (in kilometers)

## 28.632846 77.219639 Rajiv Chowk is considered the center of the city. 
# Cities of Delhi used used this same notional center.
# EPSG3857, WGS84

colonies['ndmc_dist'] = 0
ndmc = Point(28.632846, 77.219639)
for idx, row in colonies.iterrows():
    colonies.loc[idx, 'ndmc_dist'] = ndmc.distance(row['centroid'])/1000.0

In [14]:
# Remove centroid and polygon_neighbors columns, which
# are no longer needed now that we have nbr_dist
colonies = colonies.drop(columns=['centroid', 'polygon_neighbors'])

In [15]:
colonies.head(2)

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874


In [16]:
## Save `colonies` as pickle file
with open('colonies_for_psi.data', 'wb') as f:
    pickle.dump(colonies, f)

## START HERE: Load colonies GeoDataFrame for PSI

In [ ]:
with open('colonies_for_psi.data', 'rb') as f:
    colonies = pickle.load(f)

In [ ]:
colonies.head(2)

## Load Ration Shop and check for missing geometries or duplicate rows

In [17]:
# Read in ration shop data
ration_shops = gpd.read_file('RationShops.shp')

# Make sure that all ration shops have a geometry
ration_shops[ration_shops['geometry'] == None]

,S No.,District,License No,FPS ID,Circle,FPS Shop N,Address Of,Contact No,Latitude,Longitude,geometry


In [18]:
# Check for duplicate rows
ration_shops[ration_shops.duplicated()]

,S No.,District,License No,FPS ID,Circle,FPS Shop N,Address Of,Contact No,Latitude,Longitude,geometry


## TODO: 
* Check for geometry duplicates
* Check that all geometries are in Delhi
* Create function `check_shapefile`
* Duplicates
    * be more selected in checking for duplicates? Check for most minimal things
    * be restricted in what I am checking out - map no, reg no, and geometry.
    * print this out...
* Unauthorized colonies

## Convert ration shop index creation into function 

In [19]:
colonies = spatial_index_utils.create_service_index(polygon_gdf=colonies, 
                                                    point_gdf=ration_shops, 
                                                    service_name="ration", 
                                                    epsg_code=epsg_code)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [20]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist,ration_idx
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...","[(2082, 1.1670172138500838), (1148, 1.34908887...",6984.200040,1437.126828,0.000556
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...","[(1760, 0.5687093143498951), (2082, 1.26964464...",27286.639479,1438.223487,0.000696
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...","[(1276, 1.1670172138500838), (1528, 1.26964464...",30131.546842,1437.799997,0.000548


## Load in Schools Dataset and Generate Index

In [21]:
# Read in schools data
schools = gpd.read_file('DelhiSchoolsMerged.shp')

# Make sure that all ration shops have a geometry
schools[schools['geometry'] == None]

,objectid_1,objectid,vilname,schname,schcd,schcat,school_cat,pincode,rururb,location,...,school_typ,schmgt,management,dtname,stname,stcode11,dtcode11,sdtcode11,sdtname,geometry


In [22]:
# Check for duplicate rows
schools[schools.duplicated()]

,objectid_1,objectid,vilname,schname,schcd,schcat,school_cat,pincode,rururb,location,...,school_typ,schmgt,management,dtname,stname,stcode11,dtcode11,sdtcode11,sdtname,geometry


In [23]:
colonies = spatial_index_utils.create_service_index(polygon_gdf=colonies, 
                                                    point_gdf=schools, 
                                                    service_name="school", 
                                                    epsg_code=epsg_code)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [24]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist,ration_idx,school_idx
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814,0.010111
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628,0.017250
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...","[(2082, 1.1670172138500838), (1148, 1.34908887...",6984.200040,1437.126828,0.000556,0.006832
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...","[(1760, 0.5687093143498951), (2082, 1.26964464...",27286.639479,1438.223487,0.000696,0.003634
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...","[(1276, 1.1670172138500838), (1528, 1.26964464...",30131.546842,1437.799997,0.000548,0.002687


## Convert CSV data into shapefiles

In [25]:
# Delhi constants
lat_min = 28.412593
lat_max = 28.881338
lon_min = 76.83806899999999
lon_max = 77.3484578

In [26]:
delhi_bbox = box(lon_min, lat_min, lon_max, lat_max)

### ATM dataset

In [27]:
atm = gpd.read_file('atm_wgs84.shp')

In [28]:
atm.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [29]:
len(atm)

8226

In [30]:
atm = atm[atm.intersects(delhi_bbox)]

In [31]:
len(atm)

8215

In [32]:
# Make sure that all have a geometry
atm[atm['geometry'] == None]

,objectid,Bank_Type,bank_name,bk_corp_bc,Lattitude,Longitude,geometry


In [33]:
# Check for duplicate rows
atm[atm.duplicated()]

,objectid,Bank_Type,bank_name,bk_corp_bc,Lattitude,Longitude,geometry


In [34]:
colonies = spatial_index_utils.create_service_index(polygon_gdf=colonies, 
                                                    point_gdf=atm, 
                                                    service_name="atm", 
                                                    epsg_code=epsg_code)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [35]:
colonies.head(2)

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist,ration_idx,school_idx,atm_idx
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814,0.010111,0.000034
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628,0.017250,0.000193


In [36]:
colonies['atm_idx'].max()

1.0

### Bank Index

In [37]:
bank = gpd.read_file('bank_wgs84.shp')

In [38]:
bank.head()

,objectid,bank_name,bank_cd,Latitude,Longitude,field_6,geometry
0,138188,Rural Cooperative Bank,NRCB,28.436036,77.175627,NaN,POINT (77.17563 28.43604)
1,138229,Punjab National Bank,PUNB,28.440588,77.165772,NaN,POINT (77.16577 28.44059)
2,138358,Canara Bank,CNRB,28.452690,77.152180,NaN,POINT (77.15218 28.45269)
3,138386,HDFC Bank Ltd,HDFC,28.454200,77.168000,NaN,POINT (77.16800 28.45420)
4,138403,Canara Bank,CNRB,28.455020,77.183700,NaN,POINT (77.18370 28.45502)


In [39]:
bank.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [40]:
#bank = bank[bank.intersects(delhi_bbox)]

In [41]:
# Make sure that all have a geometry
bank[bank['geometry'] == None]

,objectid,bank_name,bank_cd,Latitude,Longitude,field_6,geometry


In [42]:
bank[bank.duplicated()]

,objectid,bank_name,bank_cd,Latitude,Longitude,field_6,geometry


In [43]:
colonies = spatial_index_utils.create_service_index(polygon_gdf=colonies, 
                                                    point_gdf=bank, 
                                                    service_name="bank", 
                                                    epsg_code=epsg_code)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [44]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist,ration_idx,school_idx,atm_idx,bank_idx
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814,0.010111,0.000034,0.000029
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628,0.017250,0.000193,0.000186
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...","[(2082, 1.1670172138500838), (1148, 1.34908887...",6984.200040,1437.126828,0.000556,0.006832,0.000056,0.000049
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...","[(1760, 0.5687093143498951), (2082, 1.26964464...",27286.639479,1438.223487,0.000696,0.003634,0.000044,0.000051
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...","[(1276, 1.1670172138500838), (1528, 1.26964464...",30131.546842,1437.799997,0.000548,0.002687,0.000030,0.000034


### Metro

In [45]:
metro = gpd.read_file('metro_wgs84.shp')

In [46]:
metro.head()

,S #,Metro Stat,Latitude,Longitude,geometry
0,1,Dilshad Garden Metro Station,28.675898,77.321517,POINT (77.32152 28.67590)
1,2,Jhilmil Metro Station,28.675773,77.312494,POINT (77.31249 28.67577)
2,3,Mansarovar Park Metro Station,28.675566,77.300119,POINT (77.30012 28.67557)
3,4,Shahadra Metro Station,28.673475,77.290076,POINT (77.29008 28.67347)
4,5,Welcome Metro Station,28.671790,77.277681,POINT (77.27768 28.67179)


In [47]:
# Make sure that all have a geometry
metro[metro['geometry'] == None]

,S #,Metro Stat,Latitude,Longitude,geometry


In [48]:
# Make sure none are duplicated
metro[metro.duplicated()]

,S #,Metro Stat,Latitude,Longitude,geometry


In [49]:
colonies = spatial_index_utils.create_service_index(polygon_gdf=colonies, 
                                                    point_gdf=metro, 
                                                    service_name="metro", 
                                                    epsg_code=epsg_code)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [50]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist,ration_idx,school_idx,atm_idx,bank_idx,metro_idx
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814,0.010111,0.000034,0.000029,0.0
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628,0.017250,0.000193,0.000186,0.0
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...","[(2082, 1.1670172138500838), (1148, 1.34908887...",6984.200040,1437.126828,0.000556,0.006832,0.000056,0.000049,0.0
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...","[(1760, 0.5687093143498951), (2082, 1.26964464...",27286.639479,1438.223487,0.000696,0.003634,0.000044,0.000051,0.0
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...","[(1276, 1.1670172138500838), (1528, 1.26964464...",30131.546842,1437.799997,0.000548,0.002687,0.000030,0.000034,0.0


### Police

In [51]:
police = gpd.read_file('police_wgs84.shp')

In [52]:
police.head()

,NAME,POLICE_STA,DISTRICT,FID,Station,Latitude,Longitude,layer,path,geometry
0,PS CIVIL LINES,CIVIL LINES,NORTH,NaN,None,NaN,NaN,Police_Station,C:/Users/bwbel/Downloads/Public Services-20200...,POINT (77.22172 28.68900)
1,PS TIMAR PUR,TIMAR PUR,NORTH,NaN,None,NaN,NaN,Police_Station,C:/Users/bwbel/Downloads/Public Services-20200...,POINT (77.22440 28.70679)
2,PS ROOP NAGAR,ROOP NAGAR,NORTH,NaN,None,NaN,NaN,Police_Station,C:/Users/bwbel/Downloads/Public Services-20200...,POINT (77.20254 28.68486)
3,PS SARAI ROHILLA,SARAI ROHILLA,NORTH,NaN,None,NaN,NaN,Police_Station,C:/Users/bwbel/Downloads/Public Services-20200...,POINT (77.18357 28.66875)
4,PS BARA HINDU RAO,BARA HINDU RAO,NORTH,NaN,None,NaN,NaN,Police_Station,C:/Users/bwbel/Downloads/Public Services-20200...,POINT (77.20814 28.66568)


In [53]:
# Make sure that all have a geometry
police[police['geometry'] == None]

,NAME,POLICE_STA,DISTRICT,FID,Station,Latitude,Longitude,layer,path,geometry


In [54]:
# Make sure none are duplicated
police[police.duplicated()]

,NAME,POLICE_STA,DISTRICT,FID,Station,Latitude,Longitude,layer,path,geometry


In [55]:
colonies = spatial_index_utils.create_service_index(polygon_gdf=colonies, 
                                                    point_gdf=police, 
                                                    service_name="police", 
                                                    epsg_code=epsg_code)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [56]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist,ration_idx,school_idx,atm_idx,bank_idx,metro_idx,police_idx
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814,0.010111,0.000034,0.000029,0.0,0.000000
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628,0.017250,0.000193,0.000186,0.0,0.000429
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...","[(2082, 1.1670172138500838), (1148, 1.34908887...",6984.200040,1437.126828,0.000556,0.006832,0.000056,0.000049,0.0,0.000000
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...","[(1760, 0.5687093143498951), (2082, 1.26964464...",27286.639479,1438.223487,0.000696,0.003634,0.000044,0.000051,0.0,0.000046
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...","[(1276, 1.1670172138500838), (1528, 1.26964464...",30131.546842,1437.799997,0.000548,0.002687,0.000030,0.000034,0.0,0.000000


### Bus

In [57]:
bus = gpd.read_file('bus.shp')

In [58]:
bus.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,geometry
0,0,None,Adarsh Nagar / Bharola Village,28.715917,77.170867,POINT (77.17087 28.71592)
1,1,None,British High Comission,28.598533,77.191383,POINT (77.19138 28.59853)
2,2,None,Azad Market,28.664700,77.208400,POINT (77.20840 28.66470)
3,3,None,Kidwai Nagar,28.575700,77.209700,POINT (77.20970 28.57570)
4,4,None,Rashid Market,28.650200,77.278667,POINT (77.27867 28.65020)


In [59]:
len(bus)

3210

In [60]:
# Make sure that all have a geometry
bus[bus['geometry'] == None]

,stop_id,stop_code,stop_name,stop_lat,stop_lon,geometry


In [61]:
# Make sure none are duplicated
bus[bus.duplicated()]

,stop_id,stop_code,stop_name,stop_lat,stop_lon,geometry


In [62]:
colonies = spatial_index_utils.create_service_index(polygon_gdf=colonies, 
                                                    point_gdf=bus, 
                                                    service_name="bus", 
                                                    epsg_code=epsg_code)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [63]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist,ration_idx,school_idx,atm_idx,bank_idx,metro_idx,police_idx,bus_idx
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814,0.010111,0.000034,0.000029,0.0,0.000000,0.000000
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628,0.017250,0.000193,0.000186,0.0,0.000429,0.000081
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...","[(2082, 1.1670172138500838), (1148, 1.34908887...",6984.200040,1437.126828,0.000556,0.006832,0.000056,0.000049,0.0,0.000000,0.000057
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...","[(1760, 0.5687093143498951), (2082, 1.26964464...",27286.639479,1438.223487,0.000696,0.003634,0.000044,0.000051,0.0,0.000046,0.000029
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...","[(1276, 1.1670172138500838), (1528, 1.26964464...",30131.546842,1437.799997,0.000548,0.002687,0.000030,0.000034,0.0,0.000000,0.000031


## Add area column

In [64]:
colonies['area_m2'] = colonies['geometry'].area

In [65]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist,ration_idx,school_idx,atm_idx,bank_idx,metro_idx,police_idx,bus_idx,area_m2
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814,0.010111,0.000034,0.000029,0.0,0.000000,0.000000,3.575983e+06
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628,0.017250,0.000193,0.000186,0.0,0.000429,0.000081,2.840585e+05
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...","[(2082, 1.1670172138500838), (1148, 1.34908887...",6984.200040,1437.126828,0.000556,0.006832,0.000056,0.000049,0.0,0.000000,0.000057,1.951833e+06
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...","[(1760, 0.5687093143498951), (2082, 1.26964464...",27286.639479,1438.223487,0.000696,0.003634,0.000044,0.000051,0.0,0.000046,0.000029,1.535259e+06
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...","[(1276, 1.1670172138500838), (1528, 1.26964464...",30131.546842,1437.799997,0.000548,0.002687,0.000030,0.000034,0.0,0.000000,0.000031,3.270311e+06


## Rename `ndmc_dist` to `ndmc_dist_km` to reflect unit

In [71]:
colonies = colonies.rename(columns={'ndmc_dist':'ndmc_dist_km'})

In [72]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,nbr_dist,population,ndmc_dist_km,ration_idx,school_idx,atm_idx,bank_idx,metro_idx,police_idx,bus_idx,area_m2
0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...","[(2183, 2.0167464333325045), (1148, 1.90525373...",4415.586042,1440.048891,0.000814,0.010111,0.000034,0.000029,0.0,0.000000,0.000000,3.575983e+06
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...","[(1528, 0.5687093143498951), (2869, 0.60112926...",4547.507628,1437.957874,0.002628,0.017250,0.000193,0.000186,0.0,0.000429,0.000081,2.840585e+05
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...","[(2082, 1.1670172138500838), (1148, 1.34908887...",6984.200040,1437.126828,0.000556,0.006832,0.000056,0.000049,0.0,0.000000,0.000057,1.951833e+06
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...","[(1760, 0.5687093143498951), (2082, 1.26964464...",27286.639479,1438.223487,0.000696,0.003634,0.000044,0.000051,0.0,0.000046,0.000029,1.535259e+06
4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...","[(1276, 1.1670172138500838), (1528, 1.26964464...",30131.546842,1437.799997,0.000548,0.002687,0.000030,0.000034,0.0,0.000000,0.000031,3.270311e+06


## Save ration shop index (and descriptive data) to CSV and pickle files

In [73]:
colonies.to_csv('colonies_psi_25july.csv')

In [74]:
with open('colonies_psi_25july.data', 'wb') as f:
    pickle.dump(colonies, f)